In [1]:
import pandas as pd
import numpy as np

In [2]:
data_folder = "C:\\Users\\spars\\Documents\\Master\\JHU\TML\\HomePriceBeastNew\\"

In [3]:
merged_home_data_time_series = pd.read_csv(f"{data_folder}merged_home_data_time_series.csv", low_memory=True, thousands=',')

In [4]:
merged_home_data_time_series.columns

Index(['state_code', 'county_name', 'period_begin', 'inventory',
       'inventory_lag_1', 'inventory_lag_2', 'inventory_lag_3',
       'inventory_lag_4', 'inventory_lag_5', 'inventory_lead_1',
       'inventory_lead_2', 'inventory_lead_3', 'median_sale_price',
       'median_sale_price_lag_1', 'median_sale_price_lag_2',
       'median_sale_price_lag_3', 'median_sale_price_lag_4',
       'median_sale_price_lag_5', 'week_num', 'week_num_lag_1',
       'week_num_lag_2', 'week_num_lag_3', 'week_num_lag_4', 'week_num_lag_5',
       'month', 'month_lag_1', 'month_lag_2', 'month_lag_3', 'month_lag_4',
       'month_lag_5', 'week_offset', 'week_offset_lag_1', 'week_offset_lag_2',
       'week_offset_lag_3', 'week_offset_lag_4', 'week_offset_lag_5',
       'R_INTERNATIONAL_MIG_2019', 'Unemployment_rate_2020',
       'PCT_COLL_4_2015_19', 'PCT_COLL_1TO3_2000', 'PCT_HSD_Only_2000',
       'R_NET_MIG_2019', 'Med_HH_Income_Percent_of_State_Total_2019',
       'GQ_ESTIMATES_2019', 'N_POP_CHG_2019',

In [5]:
def transform_types_and_chunk(data, non_float_cols, date_col, dummy_cols):
    for x in data.columns:
        if x not in non_float_cols:
            data[x] = pd.to_numeric(data[x])

    data = pd.get_dummies(data, columns = dummy_cols)

    data[date_col] = pd.to_datetime(data[date_col])
    post_covid_frame = data[data[date_col] >= "2020-03-01"]
    train_frame = data[data[date_col] < "2019-12-01"]
    test_frame = data[((data[date_col] >= "2019-12-01") & (data[date_col] < "2020-03-01"))]
    
    return train_frame, test_frame, post_covid_frame

In [6]:
non_float_cols = ["state_code", "county_name", "period_begin", "state_code_dummy"]

#convert these columns to encoding. 
dummy_cols = ["state_code_dummy"]

#Keep original state code data after dummifyto

merged_home_data_time_series['state_code_dummy'] = merged_home_data_time_series['state_code']

train_frame, test_frame, post_covid_frame = transform_types_and_chunk(merged_home_data_time_series,
                                 non_float_cols,
                                 "period_begin",
                                 dummy_cols)

In [7]:
train_frame.columns

Index(['state_code', 'county_name', 'period_begin', 'inventory',
       'inventory_lag_1', 'inventory_lag_2', 'inventory_lag_3',
       'inventory_lag_4', 'inventory_lag_5', 'inventory_lead_1',
       ...
       'state_code_dummy_RI', 'state_code_dummy_SC', 'state_code_dummy_TN',
       'state_code_dummy_TX', 'state_code_dummy_UT', 'state_code_dummy_VA',
       'state_code_dummy_VT', 'state_code_dummy_WA', 'state_code_dummy_WI',
       'state_code_dummy_WV'],
      dtype='object', length=118)

In [8]:
post_covid_frame[['state_code', 'county_name', 'period_begin', 'inventory']].to_csv(f"{data_folder}post_covid_subset_frame.csv")

In [9]:
test_frame[['state_code', 'county_name', 'period_begin', 'inventory']].to_csv(f"{data_folder}test_subset_frame.csv")

In [10]:
def convert_frame_to_numpy(df, remove_cols, target_prefix, related_prefixes, J,H):
    
    #assemble lag variables.
    y_lag_cols = [f'{target_prefix}_lag_{j}' for j in range(J,0,-1)]
    y_lead_cols = [target_prefix] + [f'{target_prefix}_lead_{h}' for h in range(1,H+1,1)]    
    
    flat_drop = []
    x_rel_cols = []
    for related_prefix in related_prefixes:
        curr_prefix = [f'{related_prefix}_lag_{j}' for j in range(J,0,-1)]
        x_rel_cols.append(curr_prefix)
        flat_drop = flat_drop + curr_prefix
        
    other_cols = [x for x in df.columns if x not in y_lag_cols\
                  + y_lead_cols + flat_drop + remove_cols + related_prefixes]
    
    print(f"Length of other columns = {len(other_cols)}")
    print(other_cols)
    
    def get_label_row(row):
        label = np.array([row[remove_cols].values])
        
        return label
    
    def get_xvec_row(row):
        x = np.array([row[y_lag_cols].values])
        
        #Removing sale price in inventory models 
        for x_rel in x_rel_cols:
            x = np.append(x,[row[x_rel].values],axis=0)
        
        stat_val = row[other_cols].values
        stat_val = np.tile(stat_val,[J,1])
        stat_val = np.transpose(stat_val)
        x = np.append(x,stat_val,axis=0)
        
        return x
    
    def get_yvec_row(row):
        y = np.array([row[y_lead_cols].values])
        
        return y
            
    X = np.array(df.apply(get_xvec_row, axis = 1))
    y = np.array(df.apply(get_yvec_row, axis = 1))
    label = np.array(df.apply(get_label_row, axis = 1))
    
    return X,y, label
    
remove_cols = ["county_name", "period_begin", "state_code"]
target_prefix = 'inventory'
related_prefix = ['week_num','month','week_offset']
J=5
H=3

In [11]:
X_train, y_train, train_label = convert_frame_to_numpy(train_frame, 
                                          remove_cols, 
                                          target_prefix, 
                                          related_prefix, 
                                          J,H)

Length of other columns = 88
['median_sale_price', 'median_sale_price_lag_1', 'median_sale_price_lag_2', 'median_sale_price_lag_3', 'median_sale_price_lag_4', 'median_sale_price_lag_5', 'R_INTERNATIONAL_MIG_2019', 'Unemployment_rate_2020', 'PCT_COLL_4_2015_19', 'PCT_COLL_1TO3_2000', 'PCT_HSD_Only_2000', 'R_NET_MIG_2019', 'Med_HH_Income_Percent_of_State_Total_2019', 'GQ_ESTIMATES_2019', 'N_POP_CHG_2019', 'INTERNATIONAL_MIG_2019', 'NET_MIG_2019', 'HSD_Only_2000', 'DOMESTIC_MIG_2019', 'RESIDUAL_2019', 'Deaths_2019', 'COLL_4_2000', 'POP_ESTIMATE_2019', 'LT_HSD_2015_19', 'COLL_1TO3_2000', 'Unemployed_2020', 'NATURAL_INC_2019', 'GQ_ESTIMATES_BASE_2010', 'Employed_2020', 'LT_HSD_2000', 'COLL_4_2015_19', 'HSD_Only_2015_19', 'COLL_1TO3_2015_19', 'Civilian_labor_force_2020', 'CENSUS_2010_POP', 'PCT_LT_HSD_2000', 'R_birth_2019', 'PCT_COLL_1TO3_2015_19', 'PCT_COLL_4_2000', 'Economic_typology_2015', 'R_death_2019', 'state_code_dummy_AK', 'state_code_dummy_AL', 'state_code_dummy_AR', 'state_code_dum

In [12]:
X_test, y_test, test_label = convert_frame_to_numpy(test_frame, 
                                        remove_cols, 
                                        target_prefix, 
                                        related_prefix, 
                                        J,H)

Length of other columns = 88
['median_sale_price', 'median_sale_price_lag_1', 'median_sale_price_lag_2', 'median_sale_price_lag_3', 'median_sale_price_lag_4', 'median_sale_price_lag_5', 'R_INTERNATIONAL_MIG_2019', 'Unemployment_rate_2020', 'PCT_COLL_4_2015_19', 'PCT_COLL_1TO3_2000', 'PCT_HSD_Only_2000', 'R_NET_MIG_2019', 'Med_HH_Income_Percent_of_State_Total_2019', 'GQ_ESTIMATES_2019', 'N_POP_CHG_2019', 'INTERNATIONAL_MIG_2019', 'NET_MIG_2019', 'HSD_Only_2000', 'DOMESTIC_MIG_2019', 'RESIDUAL_2019', 'Deaths_2019', 'COLL_4_2000', 'POP_ESTIMATE_2019', 'LT_HSD_2015_19', 'COLL_1TO3_2000', 'Unemployed_2020', 'NATURAL_INC_2019', 'GQ_ESTIMATES_BASE_2010', 'Employed_2020', 'LT_HSD_2000', 'COLL_4_2015_19', 'HSD_Only_2015_19', 'COLL_1TO3_2015_19', 'Civilian_labor_force_2020', 'CENSUS_2010_POP', 'PCT_LT_HSD_2000', 'R_birth_2019', 'PCT_COLL_1TO3_2015_19', 'PCT_COLL_4_2000', 'Economic_typology_2015', 'R_death_2019', 'state_code_dummy_AK', 'state_code_dummy_AL', 'state_code_dummy_AR', 'state_code_dum

In [13]:
X_post_covid, y_post_covid, post_covid_label = convert_frame_to_numpy(post_covid_frame, 
                                        remove_cols, 
                                        target_prefix, 
                                        related_prefix, 
                                        J,H)

Length of other columns = 88
['median_sale_price', 'median_sale_price_lag_1', 'median_sale_price_lag_2', 'median_sale_price_lag_3', 'median_sale_price_lag_4', 'median_sale_price_lag_5', 'R_INTERNATIONAL_MIG_2019', 'Unemployment_rate_2020', 'PCT_COLL_4_2015_19', 'PCT_COLL_1TO3_2000', 'PCT_HSD_Only_2000', 'R_NET_MIG_2019', 'Med_HH_Income_Percent_of_State_Total_2019', 'GQ_ESTIMATES_2019', 'N_POP_CHG_2019', 'INTERNATIONAL_MIG_2019', 'NET_MIG_2019', 'HSD_Only_2000', 'DOMESTIC_MIG_2019', 'RESIDUAL_2019', 'Deaths_2019', 'COLL_4_2000', 'POP_ESTIMATE_2019', 'LT_HSD_2015_19', 'COLL_1TO3_2000', 'Unemployed_2020', 'NATURAL_INC_2019', 'GQ_ESTIMATES_BASE_2010', 'Employed_2020', 'LT_HSD_2000', 'COLL_4_2015_19', 'HSD_Only_2015_19', 'COLL_1TO3_2015_19', 'Civilian_labor_force_2020', 'CENSUS_2010_POP', 'PCT_LT_HSD_2000', 'R_birth_2019', 'PCT_COLL_1TO3_2015_19', 'PCT_COLL_4_2000', 'Economic_typology_2015', 'R_death_2019', 'state_code_dummy_AK', 'state_code_dummy_AL', 'state_code_dummy_AR', 'state_code_dum

In [14]:
stack_range = np.array(X_post_covid[0]).shape[0]
print(f"Stack Range : {stack_range}")

Stack Range : 92


In [15]:
def chunks(a, size):
    arr = iter(a)
    for v in arr:
        tmp = [ v ]
        for i,v in zip( range( size - 1 ), arr ):
            tmp.append( v )
        yield tmp

In [16]:
X_train_stack = list(chunks(np.vstack(X_train), stack_range))
X_test_stack = list(chunks(np.vstack(X_test), stack_range))
label_train_stack = np.expand_dims(np.vstack(train_label),axis=2)
label_test_stack = np.expand_dims(np.vstack(test_label),axis=2)
y_train_stack = np.expand_dims(np.vstack(y_train),axis=2)
y_test_stack = np.expand_dims(np.vstack(y_test),axis=2)

In [17]:
X_train_swap = np.array(X_train_stack).swapaxes(0,1).swapaxes(0,2)
X_test_swap = np.array(X_test_stack).swapaxes(0,1).swapaxes(0,2)
label_train_swap = np.array(label_train_stack).swapaxes(0,1)
label_test_swap = np.array(label_test_stack).swapaxes(0,1)
y_train_swap = np.array(y_train_stack).swapaxes(0,1)
y_test_swap = np.array(y_test_stack).swapaxes(0,1)

In [18]:
X_post_covid_stack = list(chunks(np.vstack(X_post_covid), stack_range))
label_post_covid_stack = np.expand_dims(np.vstack(post_covid_label),axis=2)
y_post_covid_stack = np.expand_dims(np.vstack(y_post_covid),axis=2)

X_post_covid_swap = np.array(X_post_covid_stack).swapaxes(0,1).swapaxes(0,2)
label_post_covid_swap = np.array(label_post_covid_stack).swapaxes(0,1)
y_post_covid_swap = np.array(y_post_covid_stack).swapaxes(0,1)

In [19]:
with open(f"{data_folder}all_model_data.npy", 'wb') as f:
    np.save(f, X_train_swap.astype(float))
    np.save(f, y_train_swap.astype(float))
    np.save(f, X_test_swap.astype(float))
    np.save(f, y_test_swap.astype(float))

In [20]:
with open(f"{data_folder}all_model_labels_mapping.npy", 'wb') as f:
    np.save(f, label_train_swap)
    np.save(f, label_test_swap)

In [21]:
with open(f"{data_folder}post_covid_inv_data.npy", 'wb') as f:
    np.save(f, X_post_covid_swap.astype(float))
    np.save(f, y_post_covid_swap.astype(float))

In [22]:
with open(f"{data_folder}post_covid_inv_labels_mapping.npy", 'wb') as f:
    np.save(f, label_post_covid_swap)

In [23]:
X_train_swap[:,0,0].shape

(5,)

In [24]:
X_post_covid_swap.shape

(5, 118263, 92)